In [150]:
import warnings
warnings.filterwarnings('ignore')

In [151]:
import dotenv
import os
dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [152]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [153]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",convert_system_message_to_human=False)

In [154]:
import bs4
from langchain import hub

In [155]:
from langchain.chains import create_retrieval_chain

In [156]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma

In [157]:
from langchain_community.document_loaders import PyPDFLoader

In [158]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder

In [159]:
loader = PyPDFLoader('D:\Hackathons\SAP-HAkathon\Fraud Suraksha\GMaps.pdf')

In [160]:
doc = loader.load()

In [161]:
doc

[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.4', 'creationdate': '2025-03-27T04:29:32+00:00', 'title': 'Markdown To PDF', 'source': 'D:\\Hackathons\\SAP-HAkathon\\Fraud Suraksha\\GMaps.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Detailed Report: Google Maps Scam (Task-Based Fraud) Detailed Report: Google Maps Scam (Task-Based Fraud)\n1. Introduction\n1. Introduction\nThis report outlines a fraudulent scheme involving fake Google Maps review tasks, where victims are lured into performing small tasks for payment before being\ntricked into sending money under the guise of "merchant tasks." The scam operates through WhatsApp and Telegram, targeting individuals with promises of easy\nearnings.\n2. How the Scam Operates\n2. How the Scam Operates\nStep 1: Initial Contact\nStep 1: Initial Contact\nThe victim receives a message on \nWhatsApp\nWhatsApp\n from an unknown number.\nThe scammer offers payment in exchange for \nwriting Google Maps 

In [162]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [163]:
splits

[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.4', 'creationdate': '2025-03-27T04:29:32+00:00', 'title': 'Markdown To PDF', 'source': 'D:\\Hackathons\\SAP-HAkathon\\Fraud Suraksha\\GMaps.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Detailed Report: Google Maps Scam (Task-Based Fraud) Detailed Report: Google Maps Scam (Task-Based Fraud)\n1. Introduction\n1. Introduction\nThis report outlines a fraudulent scheme involving fake Google Maps review tasks, where victims are lured into performing small tasks for payment before being\ntricked into sending money under the guise of "merchant tasks." The scam operates through WhatsApp and Telegram, targeting individuals with promises of easy\nearnings.\n2. How the Scam Operates\n2. How the Scam Operates\nStep 1: Initial Contact\nStep 1: Initial Contact\nThe victim receives a message on \nWhatsApp\nWhatsApp\n from an unknown number.\nThe scammer offers payment in exchange for \nwriting Google Maps 

In [164]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [165]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000018EA493B0D0>, search_kwargs={})

In [166]:
system_prompt = (
    "You are an AI-powered fraud detection assistant designed to help users verify whether a person, message, or situation is fraudulent or safe. "
    "Use the following retrieved context to analyze scam patterns and assess the fraud risk score. "
    "If you don't have enough data to confirm, suggest caution and advise the user to be careful. "
    "Ask relevant follow-up questions if needed to gather more details before making a decision. "
    "Provide structured responses, including fraud risk score, detected red flags, and recommended actions. "
    "Never generate fake reports, financial guarantees, or unverifiable claims. "
    "\n\n"
    "{context}"
)  


In [167]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [168]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [169]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [170]:
response = rag_chain.invoke({"input":"What is the report contains"})

In [171]:
response["answer"]

'The report outlines a fraudulent scheme involving fake Google Maps review tasks. In this scam, victims are lured into performing small tasks for payment before being tricked into sending money under the guise of "merchant tasks." The scam typically operates through WhatsApp and Telegram, targeting individuals with promises of easy earnings. The initial contact is usually made via WhatsApp from an unknown number, where the scammer offers payment for writing Google Maps reviews. A small initial payment is often sent to gain the victim’s trust. The scammer then asks the victim to move the conversation to Telegram.'

In [172]:
response = rag_chain.invoke({"input":"Hii im dhanush"})

In [173]:
response["answer"]

"Hello Dhanush, how can I help you today? Are you concerned about a potential scam or suspicious activity? If so, please provide as many details as possible about the situation, including any messages or communications you've received."

In [176]:
response = rag_chain.invoke({"input":"What is my name"})
response["answer"]

'I do not have access to personally identifiable information (PII), including your name. As an AI, I have no memory of past conversations.'

### RAG With Memory

In [177]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

In [178]:
store = {}

In [179]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [180]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [1]:
config = {
        "configurable": {"session_id": "abc123"}
    }

In [ ]:
conversational_rag_chain.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

In [181]:
conversational_rag_chain.invoke(
    {"input": "What is Google Maps Fraud?"},
    config=config,  # constructs a key "abc123" in `store`.
)["answer"]

"Google Maps fraud involves scammers exploiting the platform's features by creating fake business listings to impersonate legitimate companies or inventing non-existent ones. Unsuspecting customers who interact with these fake listings may be overcharged, receive subpar services, or become victims of fraud. In March 2025, Google filed a lawsuit against individuals responsible for approximately 10,000 fraudulent business listings."

In [183]:
conversational_rag_chain.invoke(
    {"input": "My name is Dhanush"},
    config={
        "configurable": {"session_id": "session"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Okay, Dhanush. How can I help you today? Are you concerned about a potential scam, or do you want to verify the legitimacy of a person, message, or situation? Please provide as much detail as possible so I can assist you effectively.'

In [184]:
conversational_rag_chain.invoke(
    {"input": "What is my name"},
    config={
        "configurable": {"session_id": "session"}
    },  
)["answer"]

'I do not have access to personally identifiable information, including your name. As an AI, I respect your privacy.'

In [185]:
from langchain_core.messages import AIMessage

In [186]:
for message in store["session"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: My name is Dhanush

AI: Okay, Dhanush. How can I help you today? Are you concerned about a potential scam, or do you want to verify the legitimacy of a person, message, or situation? Please provide as much detail as possible so I can assist you effectively.

User: What is my name

AI: I do not have access to personally identifiable information, including your name. As an AI, I respect your privacy.



In [187]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Based on the provided context, here\'s a breakdown of common methods used in this scam:\n\n**1. Initial Contact via WhatsApp:**\n\n*   The scam usually starts with a message on WhatsApp from an unknown number.\n*   The message offers payment for simple tasks like writing Google Maps reviews.\n*   A small initial payment (e.g., ₹200) is sent to build trust.\n\n**2. Transition to Telegram:**\n\n*   The scammer asks the victim to move the conversation to Telegram, a platform that might have less stringent monitoring.\n*   On Telegram, a "task manager" assigns small tasks such as:\n    *   Liking YouTube videos.\n    *   Subscribing to channels.\n    *   Writing fake Google reviews.\n*   The victim receives a small payment for each task (e.g., ₹40).\n\n**3. Introduction of "Merchant Task":**\n\n*   After the victim completes a few initial tasks (5-6), they are told they need to complete a "merchant task" to receive their full salary.\n*   The scammer presents multiple payment plans (e.g.,

In [188]:
from langchain_core.messages import AIMessage

In [189]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Google Maps Fraud?

AI: Google Maps fraud involves scammers exploiting the platform's features by creating fake business listings to impersonate legitimate companies or inventing non-existent ones. Unsuspecting customers who interact with these fake listings may be overcharged, receive subpar services, or become victims of fraud. In March 2025, Google filed a lawsuit against individuals responsible for approximately 10,000 fraudulent business listings.

User: My name is Dhanush

AI: Okay, Dhanush. How can I help you today? Do you want to verify a person, message, or situation for potential fraud? Please provide as many details as possible so I can assist you effectively.

User: What are common ways of doing it?

AI: Based on the provided context, here's a breakdown of common methods used in this scam:

**1. Initial Contact via WhatsApp:**

*   The scam usually starts with a message on WhatsApp from an unknown number.
*   The message offers payment for simple tasks like wr

In [ ]:
store

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Hey i think i meat with a scammer...!"},
    config={"configurable": {"session_id": "session1"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Encountered with the whats app and asked me to write reviews and give rating and asked me to send screenshot"},
    config={"configurable": {"session_id": "session1"}},
)["answer"]

In [ ]:
for message in store["session1"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [190]:
conversational_rag_chain.invoke(
    {"input": "Hii im dhanush"},
    config={"configurable": {"session_id": "fraud"}},
)["answer"]

"Hello Dhanush, how can I help you today? Do you have a specific person, message, or situation you'd like me to check for potential fraud?"

In [191]:
conversational_rag_chain.invoke(
    {"input": "What is my name"},
    config={"configurable": {"session_id": "fraud"}},
)["answer"]

'I do not have access to personally identifiable information, including your name. As an AI, I respect your privacy and operate without storing personal data.'